In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from ipywidgets import widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual

In [2]:
#invoerwaarden Terence
Ppomp = 16 #kW
MaximumFreq = 90 #Hz
Apaneel = 22.35 #m^2
StartDate = '27/11/2018 11:13' #dd/mm/yyyy HH:mm
EndDate = '27/11/2018 18:13' #dd/mm/yyyy HH:mm

In [3]:
button = widgets.Button(description='Done')
display(button)

def on_button_clicked(b):
    Ppomp = pd.to_numeric(text.value)
    print('Klik')
    
button.on_click(on_button_clicked)

Button(description='Done', style=ButtonStyle())

In [4]:
Ppomp = pd.to_numeric(Ppomp)
Ppomp

16

In [5]:
#bestand importeren
df = pd.read_csv("~//Google Drive/11 - Triple Solar - Stagiaires/Stages/Jelmer/Modellen/Terence/06529517010004 Terence 2min - klein.csv", engine='python', delimiter=";", index_col=0, header=[0,1], skiprows=[2,3])

In [6]:
#COP tabel importeren in tabelvorm
df_COP = pd.read_csv("~//Google Drive/11 - Triple Solar - Stagiaires/Stages/Jelmer/Modellen/COP_tabel.csv", engine='python', delimiter=";", index_col=[0])

In [7]:
#index omzetten naar 'datetime'
df = df.set_index(pd.DatetimeIndex(df.index.values))

In [8]:
#van de comma's een punt maken
df = df.replace(',','.', regex=True)
df_COP= df_COP.replace(',','.', regex=True)

In [9]:
#van de comma's een punt maken
df_COP= df_COP.replace(',','.', regex=True)

In [10]:
df.dtypes

40004  outdoor temp.                 object
40014  hot water charging            object
43005  degree minutes                 int64
40013  hot water top                float64
40015  brine in                      object
40016  brine out                     object
40008  heat medium flow              object
40012  return temp.                  object
43136  current compr. frequency       int64
43084  electrical addition power      int64
43081  time factor                   object
dtype: object

In [11]:
df_COP

,30,35,40,45,50,55,60
-10,3.86,3.45,3.04,2.64,2.27,1.87,1.46
-5,4.85,4.32,3.83,3.38,2.99,2.57,2.17
0,5.50,4.93,4.39,3.88,3.45,3.05,2.63
5,6.49,5.73,5.07,4.46,3.95,3.51,3.09
10,7.51,6.54,5.73,5.06,4.48,3.97,3.50
15,8.27,7.14,6.23,5.46,4.81,4.27,3.77
20,9.13,7.79,6.74,5.87,5.15,4.56,4.03
25,9.99,8.41,7.21,6.25,5.45,4.82,4.25


In [12]:
def myround(x, base=5):
    return int(base * round(float(x)/base))

def createCOP(BI, MHF, df_COP):
    COP = []
    for indexName, columnName in zip(BI, MHF): 
        if not 30 <= myround(columnName) <= 60 or not -10 <= myround(indexName) <= 25:
            COP.append(None)
        else:
            COP.append(df_COP[str(myround(columnName))][myround(indexName)])
    return COP

df['','COP'] = createCOP(df['40016', 'brine out'], df['40008', 'heat medium flow'], df_COP)

In [13]:
df['40015', 'brine in'] = df['40015', 'brine in'].astype(float)
df['40008', 'heat medium flow'] = df['40008', 'heat medium flow'].astype(float)
#Tabel['COP'] = Tabel.apply(lambda x: COP_tabel[x['40008', 'heat medium flow', 'BT2']][x['40015', 'brine in', 'EB100-EP14-BT10']], axis = 1)

In [14]:
#Delta T berekenen
df['40004', 'outdoor temp.'] = df['40004', 'outdoor temp.'].astype(float)
df['40015', 'brine in'] = df['40015', 'brine in'].astype(float)
df['40016', 'brine out'] = df['40016', 'brine out'].astype(float)
df['','Delta_T'] = df[('40004', 'outdoor temp.')] - ((df[('40015', 'brine in')] + df[('40016', 'brine out')])/2)

In [15]:
#Actueel vermogen berekenen
df['','Actueel vermogen'] = df[('43136', 'current compr. frequency')].astype(float) / MaximumFreq * Ppomp
df['','Actueel vermogen'] = df['','Actueel vermogen'].round(2)

In [16]:
#U-waarde berekenen
df['','U_waarde',''] = df['','Actueel vermogen',''] * 1000 / df['','Delta_T',''] / 22.35
df['','U_waarde',''] = df['','U_waarde',''].astype(float).round(2)

KeyError: 'Key length (3) exceeds index depth (2)'

In [ ]:
#Het colum BT7 laten vallen (optioneel) en oneidig groot omzetten naar 100
#Het colum verwijderen is alleen vor Terence en met andere kan dit weg
df = df.drop([('40013', 'hot water top', 'BT7')], axis=1)
df = df.replace(np.inf, "100")

In [ ]:
#U-waardes die boven de 100 afkoppen naar 100 en van negative U-waardes een 0 maken.
df['','U_waarde',''] = df['','U_waarde',''].astype(float).where(df['','U_waarde',''].astype(float)<=100, other='100')
df['','U_waarde',''] = df['','U_waarde',''].astype(float).where(df['','U_waarde',''].astype(float)>0, other='0')

In [ ]:
#buiten temperaturr afronden naar hele getallen
df['40004', 'outdoor temp.', 'BT1'] = df['40004', 'outdoor temp.', 'BT1'].round(decimals=0).astype(int)

In [ ]:
df#['40004', 'outdoor temp.', 'BT1']

In [ ]:
#exporteren als CSV-bestand
df.to_csv("ouput.csv", sep=';')

In [ ]:
df_date = df['26/09/2018 18:13': '27/09/2018 00:13']
df_date
df_date.to_csv("ouput.csv", sep=';')

In [ ]:
df['40004', 'outdoor temp.', 'BT1'] = df['40004', 'outdoor temp.', 'BT1'].astype(float).where(df['40004', 'outdoor temp.', 'BT1'].astype(float)>-10, other=-10)
df[[('40004', 'outdoor temp.', 'BT1'), ('40014', 'hot water charging', 'BT6')]].astype(float).plot(use_index = True, kind = 'line', legend = True, sharex=True)#, subplots=True)
plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)

In [ ]:
lijstje = list(df)
lijstje

In [ ]:
#grafiek tekenen en opslaan
Frame = df[StartDate: EndDate]
Frame.drop([('43081', 'time factor', 'Unnamed: 11_level_2'), ('43005', 'degree minutes', 'Unnamed: 3_level_2')], axis=1).astype(float).plot(use_index = True, kind = 'line', legend = True, sharex=True)

plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
plt.savefig('plot.png', dpi = 100, bbox_inches='tight')
#plt.figure(figsize =[10, 10], dpi =100)

In [ ]:
print(Frame.to_string())

In [ ]:
Frame = Frame[[('40004', 'outdoor temp.', 'BT1'), ('40015', 'brine in', 'EB100-EP14-BT10'), ('40008', 'heat medium flow', 'BT2'), ('', 'COP', '')]]
Frame.astype(float).plot(use_index = True, kind = 'line', legend = True, sharex=True)#, subplots=True)
plt.legend(bbox_to_anchor=(1.04,1), borderaxespad=0)
plt.figure(figsize =[6.4, 4.8])
#plt.figure(dpi = 300)